## *[Занятие 2](https://hackmd.io/@J_qqq0PjTGK1be0341GpYA/SJZDs66fm#/ "Как Spark устроен внутри &amp; SparkSQL - HackMD") (contd.)*

https://hackmd.io/@J_qqq0PjTGK1be0341GpYA/SJZDs66fm#/

In [1]:
import findspark

findspark.init()

import pyspark

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
alexa = spark.read.csv("/data/top-1m.csv")

In [4]:
alexa.show()

+---+---------------+
|_c0|            _c1|
+---+---------------+
|  1|     google.com|
|  2|    youtube.com|
|  3|   facebook.com|
|  4|      baidu.com|
|  5|  wikipedia.org|
|  6|         qq.com|
|  7|     taobao.com|
|  8|      yahoo.com|
|  9|      tmall.com|
| 10|     amazon.com|
| 11|    twitter.com|
| 12|       sohu.com|
| 13|         jd.com|
| 14|       live.com|
| 15|    sina.com.cn|
| 16|  instagram.com|
| 17|      weibo.com|
| 18|         360.cn|
| 19|login.tmall.com|
| 20|     reddit.com|
+---+---------------+
only showing top 20 rows



In [5]:
alexa.orderBy("_c0").show()

+-------+--------------------+
|    _c0|                 _c1|
+-------+--------------------+
|      1|          google.com|
|     10|          amazon.com|
|    100|           zhihu.com|
|   1000|        turnitin.com|
|  10000|             kit.edu|
| 100000|     educateiowa.gov|
|1000000|  onlythefitness.com|
| 100001|     wagnardsoft.com|
| 100002|         myhermes.at|
| 100003|         pvcloud.com|
| 100004|descargas-vagos.c...|
| 100005|       blogvporn.com|
| 100006|          duedil.com|
| 100007|      doctorpiter.ru|
| 100008|      sachbaitap.com|
| 100009|         iwmbuzz.com|
|  10001|       37signals.com|
| 100010|        torrentz2.cc|
| 100011|       technoxyz.com|
| 100012|      warkscol.ac.uk|
+-------+--------------------+
only showing top 20 rows



In [6]:
from pyspark.sql import types

schema = types.StructType().add("rank", "integer").add("site", "string")

real_alexa = spark.read.csv("/data/top-1m.csv", schema=schema)

In [7]:
real_alexa.show()

+----+---------------+
|rank|           site|
+----+---------------+
|   1|     google.com|
|   2|    youtube.com|
|   3|   facebook.com|
|   4|      baidu.com|
|   5|  wikipedia.org|
|   6|         qq.com|
|   7|     taobao.com|
|   8|      yahoo.com|
|   9|      tmall.com|
|  10|     amazon.com|
|  11|    twitter.com|
|  12|       sohu.com|
|  13|         jd.com|
|  14|       live.com|
|  15|    sina.com.cn|
|  16|  instagram.com|
|  17|      weibo.com|
|  18|         360.cn|
|  19|login.tmall.com|
|  20|     reddit.com|
+----+---------------+
only showing top 20 rows



In [8]:
real_alexa.orderBy("rank").show()

+----+---------------+
|rank|           site|
+----+---------------+
|   1|     google.com|
|   2|    youtube.com|
|   3|   facebook.com|
|   4|      baidu.com|
|   5|  wikipedia.org|
|   6|         qq.com|
|   7|     taobao.com|
|   8|      yahoo.com|
|   9|      tmall.com|
|  10|     amazon.com|
|  11|    twitter.com|
|  12|       sohu.com|
|  13|         jd.com|
|  14|       live.com|
|  15|    sina.com.cn|
|  16|  instagram.com|
|  17|      weibo.com|
|  18|         360.cn|
|  19|login.tmall.com|
|  20|     reddit.com|
+----+---------------+
only showing top 20 rows



In [9]:
real_alexa.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- site: string (nullable = true)



In [10]:
from pyspark.sql import functions

(
    real_alexa
    .select(
        functions.explode(
            functions.split("site", "\.")
        ).alias("token")
    )
    .groupBy("token")
    .count()
    .withColumnRenamed("count", "shmount")
    .orderBy(functions.desc("shmount"))
    .show()
)

+--------+-------+
|   token|shmount|
+--------+-------+
|     com| 529951|
|     org|  54801|
|     net|  42212|
|      ru|  41774|
|      co|  35863|
|      de|  30184|
|      br|  21409|
|blogspot|  16691|
|      uk|  16631|
|      pl|  15019|
|      in|  13978|
|      ir|  12242|
|      it|  11209|
|      au|  10855|
|      jp|  10575|
|      fr|   9801|
|     gov|   9751|
|     edu|   8904|
|    info|   8335|
|      cz|   7750|
+--------+-------+
only showing top 20 rows



In [40]:
from pyspark.sql import functions

reverse_idx = (
    real_alexa
    .select(
        functions.explode(
            functions.split("site", "\.")
        ).alias("token"),
        functions.col("site"),
        functions.col("rank")
    )
    .checkpoint()
    .groupBy("token")
    .agg(
        {
            "site": "collect_set",
            "rank": "max",
        }
    )
)

reverse_idx.cache()

Py4JJavaError: An error occurred while calling o289.checkpoint.
: org.apache.spark.SparkException: Checkpoint directory has not been set in the SparkContext
	at org.apache.spark.rdd.RDD.checkpoint(RDD.scala:1554)
	at org.apache.spark.sql.Dataset.checkpoint(Dataset.scala:617)
	at org.apache.spark.sql.Dataset.checkpoint(Dataset.scala:575)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at

In [12]:
reverse_idx.where("token = 'google'").show()

+------+--------------------+---------+
| token|   collect_set(site)|max(rank)|
+------+--------------------+---------+
|google|[safety.google, d...|   953902|
+------+--------------------+---------+



In [13]:
reverse_idx.where("token = 'ya'").show()

+-----+--------------------+---------+
|token|   collect_set(site)|max(rank)|
+-----+--------------------+---------+
|   ya|[ya.no, ya.com, y...|   596444|
+-----+--------------------+---------+



In [14]:
zgrab = spark.read.json("/data/zgrab.json")

In [15]:
zgrab.show()

+---------------+--------------------+--------------------+---------------+--------------------+
|_corrupt_record|                data|               error|             ip|           timestamp|
+---------------+--------------------+--------------------+---------------+--------------------+
|           null|               [[,]]|Get http://172.65...| 172.65.100.108|2018-05-13T02:30:...|
|           null|[[, [<html style=...|                null| 192.230.66.113|2018-05-13T02:30:...|
|           null|[[, [,,, [,,,,,,,...|                null|  159.65.233.69|2018-05-13T02:30:...|
|           null|               [[,]]|Get http://168.25...| 168.255.128.19|2018-05-13T02:30:...|
|           null|[[, [<!doctype ht...|                null| 185.162.171.15|2018-05-13T02:30:...|
|           null|[[, [<html style=...|                null| 31.201.254.208|2018-05-13T02:30:...|
|           null|[[, [<!DOCTYPE ht...|                null|  151.40.76.245|2018-05-13T02:30:...|
|           null|             

In [16]:
zgrab.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- http: struct (nullable = true)
 |    |    |-- redirect_response_chain: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- body: string (nullable = true)
 |    |    |    |    |-- body_sha256: string (nullable = true)
 |    |    |    |    |-- content_length: long (nullable = true)
 |    |    |    |    |-- headers: struct (nullable = true)
 |    |    |    |    |    |-- accept_ranges: array (nullable = true)
 |    |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |    |-- access_control_allow_origin: array (nullable = true)
 |    |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |    |-- cache_control: array (nullable = true)
 |    |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |    |-- connection: array (nullable = true)
 

In [17]:
zgrab.select("ip").show()

+---------------+
|             ip|
+---------------+
| 172.65.100.108|
| 192.230.66.113|
|  159.65.233.69|
| 168.255.128.19|
| 185.162.171.15|
| 31.201.254.208|
|  151.40.76.245|
|  112.72.33.128|
| 218.145.166.91|
|   67.222.68.11|
|   45.60.107.95|
| 118.169.10.116|
|   45.121.57.70|
|  45.195.160.26|
|115.131.234.198|
|115.128.144.139|
| 68.171.217.103|
|  123.60.36.156|
|    119.32.58.2|
|  168.255.58.46|
+---------------+
only showing top 20 rows



In [25]:
from geoip2 import database

db = database.Reader(
    "/data/GeoLite2-Country_20190305/GeoLite2-Country.mmdb"
)


def country_of(ip):
    return db.country(ip).country.names["ru"]

country_of("1.1.1.1")

country_udf = functions.udf(country_of)

In [26]:
country_udf("ip")

Column<b'country_of(ip)'>

In [27]:
zgrab.select(country_udf("ip")).show()

Py4JJavaError: An error occurred while calling o117.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 30.0 failed 1 times, most recent failure: Lost task 0.0 in stage 30.0 (TID 610, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 361, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 236, in read_udfs
    arg_offsets, udf = read_single_udf(pickleSer, infile, eval_type, runner_conf)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 163, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 64, in read_command
    command = serializer._read_with_length(file)
  File "/opt/spark/python/lib/pyspark.zip/pyspar

In [29]:
class CountryOf(object):
    
    def __init__(self):
        self.db = None
    
    def __call__(self, ip):
        if self.db is None:
            self.db = database.Reader(
                "/data/GeoLite2-Country_20190305/GeoLite2-Country.mmdb"
            )
        return self.db.country(ip).country.names["ru"]

    
c = CountryOf()
c("1.1.1.1")

'Австралия'

In [38]:
country_detector = spark.sparkContext.broadcast(CountryOf())
country_detector_udf = functions.udf(country_detector.value)

zgrab.select(
    functions.col("ip"),
    country_detector_udf("ip"),
).show()

+---------------+-------------+
|             ip|CountryOf(ip)|
+---------------+-------------+
| 172.65.100.108|          США|
| 192.230.66.113|          США|
|  159.65.233.69|          США|
| 168.255.128.19|      Мексика|
| 185.162.171.15|      Испания|
| 31.201.254.208|   Нидерланды|
|  151.40.76.245|       Италия|
|  112.72.33.128|  Южная Корея|
| 218.145.166.91|  Южная Корея|
|   67.222.68.11|          США|
|   45.60.107.95|          США|
| 118.169.10.116|      Тайвань|
|   45.121.57.70|      Гонконг|
|  45.195.160.26|      Гонконг|
|115.131.234.198|    Австралия|
|115.128.144.139|    Австралия|
| 68.171.217.103|          США|
|  123.60.36.156|      Гонконг|
|    119.32.58.2|        Китай|
|  168.255.58.46|      Мексика|
+---------------+-------------+
only showing top 20 rows



In [49]:
noisy_alexa = (
    real_alexa
    .withColumn("rank", functions.col("rank") + functions.randn(seed=123))
)

noisy_alexa.show()

+------------------+---------------+
|              rank|           site|
+------------------+---------------+
| 1.001988081602008|     google.com|
|2.3276509951775273|    youtube.com|
|3.3598960244031226|   facebook.com|
| 4.380196619517471|      baidu.com|
|2.8273413279091124|  wikipedia.org|
| 5.251587454981575|         qq.com|
| 5.770762978079437|     taobao.com|
| 8.285684865534792|      yahoo.com|
|  9.64270567520388|      tmall.com|
| 9.414581134399056|     amazon.com|
|10.805247886805665|    twitter.com|
|11.596979930560071|       sohu.com|
|12.473087145486339|         jd.com|
|15.542764050766385|       live.com|
|14.907071638998694|    sina.com.cn|
| 14.98025569436436|  instagram.com|
| 18.47398111773678|      weibo.com|
|16.513389678588446|         360.cn|
| 18.87040873956046|login.tmall.com|
|22.107256186411995|     reddit.com|
+------------------+---------------+
only showing top 20 rows



In [50]:
noisy_alexa.registerTempTable("noisy")

In [52]:
spark.sql("SELECT * FROM noisy").show()

+------------------+---------------+
|              rank|           site|
+------------------+---------------+
| 1.001988081602008|     google.com|
|2.3276509951775273|    youtube.com|
|3.3598960244031226|   facebook.com|
| 4.380196619517471|      baidu.com|
|2.8273413279091124|  wikipedia.org|
| 5.251587454981575|         qq.com|
| 5.770762978079437|     taobao.com|
| 8.285684865534792|      yahoo.com|
|  9.64270567520388|      tmall.com|
| 9.414581134399056|     amazon.com|
|10.805247886805665|    twitter.com|
|11.596979930560071|       sohu.com|
|12.473087145486339|         jd.com|
|15.542764050766385|       live.com|
|14.907071638998694|    sina.com.cn|
| 14.98025569436436|  instagram.com|
| 18.47398111773678|      weibo.com|
|16.513389678588446|         360.cn|
| 18.87040873956046|login.tmall.com|
|22.107256186411995|     reddit.com|
+------------------+---------------+
only showing top 20 rows



In [60]:
spark.sql(
    """
    SELECT * FROM 
    noisy as l join noisy as r
    where ABS(l.rank - r.rank) <=5
    """
).show()

# если запустить .count() то все повесится из-за декартова произведения

+------------------+------------+------------------+-------------+
|              rank|        site|              rank|         site|
+------------------+------------+------------------+-------------+
| 1.001988081602008|  google.com| 1.001988081602008|   google.com|
| 1.001988081602008|  google.com|2.3276509951775273|  youtube.com|
| 1.001988081602008|  google.com|3.3598960244031226| facebook.com|
| 1.001988081602008|  google.com| 4.380196619517471|    baidu.com|
| 1.001988081602008|  google.com|2.8273413279091124|wikipedia.org|
| 1.001988081602008|  google.com| 5.251587454981575|       qq.com|
| 1.001988081602008|  google.com| 5.770762978079437|   taobao.com|
|2.3276509951775273| youtube.com| 1.001988081602008|   google.com|
|2.3276509951775273| youtube.com|2.3276509951775273|  youtube.com|
|2.3276509951775273| youtube.com|3.3598960244031226| facebook.com|
|2.3276509951775273| youtube.com| 4.380196619517471|    baidu.com|
|2.3276509951775273| youtube.com|2.8273413279091124|wikipedia.